In [2]:
import pandas as pd
import numpy as np

# 假設資料來源
df = pd.read_csv("all_dataset.csv")

print("🚀 開始處理資料...")

# Step 1. 移除 type 為 "接不到" 的列
initial_len = len(df)
df = df[df['type'] != '接不到']
print(f"✅ 已移除 '接不到' 的資料 (移除 {initial_len - len(df)} 筆)")

# 建立遮罩處理 getpoint_player
mask_not_last_in_rally = df.duplicated(subset=['rally_id'], keep='last')
df.loc[mask_not_last_in_rally, 'getpoint_player'] = np.nan
print("✅ 'getpoint_player' 處理完畢。")

# Step 2. 若 lose_reason 或 win_reason 包含「對手」則改為 NaN
for col in ['lose_reason', 'win_reason']:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: np.nan if isinstance(x, str) and "對手" in x else x)

# --- [New] Rename Columns Early ---
p1_name = None
p2_name = None
if not df.empty:
    try:
        unique_players = df['player'].dropna().unique()
        if len(unique_players) >= 2:
            p1_name = df.iloc[0]['player']
            p2_name = next((p for p in unique_players if p != p1_name), None)
            
            if p1_name and p2_name:
                rename_map = {
                    'player_score': f'{p1_name}_score',
                    'opponent_score': f'{p2_name}_score'
                }
                if 'player_score' in df.columns:
                    df = df.rename(columns=rename_map)
                    print(f"✅ [Pre-process] 已將分數欄位命名: {rename_map}")
    except Exception as e:
        print(f"⚠️ Pre-process renaming failed: {e}")

# Step 3. 每場比賽的每個 set 最後插入一筆新資料，並新增 opponent 欄位
dfs_to_concat = []

for (match_id, set_id), group in df.groupby(['match_id', 'set'], sort=False):
    group = group.copy()

    # --- 處理 opponent 欄位 ---
    players = group['player'].dropna().unique()
    if len(players) == 2:
        p1, p2 = players[0], players[1]
        opponent_map = {p1: p2, p2: p1}
        group['opponent'] = group['player'].map(opponent_map)
    else:
        group['opponent'] = np.nan
    
    dfs_to_concat.append(group)

    # 處理最後一筆 & 加分
    try:
        last_row = group.iloc[-1].copy()
        winner = last_row.get('getpoint_player')
        
        if pd.notna(winner):
            # [User Request]: 根據 winner 是誰讓 {name}_score 欄位 +1
            score_col = f"{winner}_score"
            
            if score_col in last_row:
                try:
                    last_row[score_col] = float(last_row[score_col]) + 1
                except:
                        last_row[score_col] = 1.0
            else:
                    pass
                    print(f"Warning: Score column '{score_col}' not found.")

            dfs_to_concat.append(pd.DataFrame([last_row]))
    
    except Exception as e:
        print(f"Error processing last row: {e}")
        pass

# 4. 最後一次合併
new_df = pd.concat(dfs_to_concat, ignore_index=True)

if 'score_status' in new_df.columns:
    new_df = new_df.drop(columns=['score_status'])
    print("✅ 已成功刪除 'score_status' 欄位。")

# 輸出 CSV
new_df.to_csv("processed_new_3.csv", index=False)
print("✅ 已完成：資料處理並儲存至 processed_new_3.csv")

In [3]:
import pandas as pd
import sqlite3

# 讀取 CSV 檔
csv_file = "processed_new_3.csv"
df = pd.read_csv(csv_file)

# 建立 SQLite 資料庫（若不存在會自動建立）
db_file = "processed_new_3.db"
conn = sqlite3.connect(db_file)

# 將 DataFrame 寫入 SQLite，表格名稱可自訂
table_name = "match_data"
df.to_sql(table_name, conn, if_exists="replace", index=False)

# 確認寫入完成
print(f"✅ 已將 {csv_file} 匯入 SQLite 資料庫 {db_file}，表格名稱：{table_name}")

# （可選）查看前 5 筆資料確認
print(pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5;", conn))

# 關閉連線
conn.close()
